# Using RAG with Google Analytics

Using [LangChain](https://www.langchain.com/), [GPT4All](https://gpt4all.io/index.html), and [Google Analysis Data API](https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries). Make sure to read their documentation before starting.

## Steps
1. Setup Google Analytics API.
2. Save Google Analytics Data into a Dataframe.
3. Use the Dataframe with the LLM.

### 1. Setup Google Analytics API.

You need to create a `service account` from Google Cloud and download the service key and use it's path in `key_path`. You can learn how [here](https://support.google.com/a/answer/7378726?hl=en).

In [2]:
from google.oauth2 import service_account
import pandas as pd
import itertools

key_path="/path/to/your/service/key"
credentials = service_account.Credentials.from_service_account_file(key_path)

Then, initialize Google Analytics Data API, with your preferences. Here, I am fetching countries and their corresponding sessions.

 Don't forget to set `property_id` to the project your are fetching.

In [4]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
)
client = BetaAnalyticsDataClient(credentials=credentials)
property_id = ""
starting_date = "8daysAgo"
ending_date = "yesterday"

request_api = RunReportRequest(
    property=f"properties/{property_id}",
    dimensions=[
        Dimension(name="country")
        ],
        metrics=[
            Metric(name="sessions")
        ],
        date_ranges=[DateRange(start_date=starting_date, end_date=ending_date)],
    )
response = client.run_report(request_api)
print(response)

dimension_headers {
  name: "country"
}
metric_headers {
  name: "sessions"
  type_: TYPE_INTEGER
}
rows {
  dimension_values {
    value: "Algeria"
  }
  metric_values {
    value: "84"
  }
}
rows {
  dimension_values {
    value: "Morocco"
  }
  metric_values {
    value: "52"
  }
}
rows {
  dimension_values {
    value: "Iraq"
  }
  metric_values {
    value: "44"
  }
}
rows {
  dimension_values {
    value: "Saudi Arabia"
  }
  metric_values {
    value: "44"
  }
}
rows {
  dimension_values {
    value: "Egypt"
  }
  metric_values {
    value: "42"
  }
}
rows {
  dimension_values {
    value: "Jordan"
  }
  metric_values {
    value: "28"
  }
}
rows {
  dimension_values {
    value: "Libya"
  }
  metric_values {
    value: "22"
  }
}
rows {
  dimension_values {
    value: "Yemen"
  }
  metric_values {
    value: "22"
  }
}
rows {
  dimension_values {
    value: "Tunisia"
  }
  metric_values {
    value: "15"
  }
}
rows {
  dimension_values {
    value: "Lebanon"
  }
  metric_values

### 2. Save Google Analytics Data into a csv file.
This function transform the response into a CSV file using pandas.

In [6]:
def query_data(api_response):
    dimension_headers = [header.name for header in api_response.dimension_headers]
    metric_headers = [header.name for header in api_response.metric_headers]
    dimensions = []
    metrics = []
    for i in range(len(dimension_headers)):
        dimensions.append([row.dimension_values[i].value for row in api_response.rows])
    dimensions
    for i in range(len(metric_headers)):
        metrics.append([row.metric_values[i].value for row in api_response.rows])
    headers = dimension_headers, metric_headers
    headers = list(itertools.chain.from_iterable(headers))   
    data = dimensions, metrics
    data = list(itertools.chain.from_iterable(data))
    df = pd.DataFrame(data)
    df = df.transpose()
    df.columns = headers
    return df

Saving the csv into a file

In [9]:
df = query_data(response)
df.to_csv('file.csv', index=False)
print(df)

                 country sessions
0                Algeria       84
1                Morocco       52
2                   Iraq       44
3           Saudi Arabia       44
4                  Egypt       42
5                 Jordan       28
6                  Libya       22
7                  Yemen       22
8                Tunisia       15
9                Lebanon       14
10                 Syria       12
11                Israel       11
12                 Sudan        8
13                  Oman        4
14             Singapore        4
15  United Arab Emirates        4
16               Bahrain        3
17               Austria        2
18               Germany        2
19                  Chad        1
20                France        1
21                Kuwait        1
22            Mauritania        1
23           Netherlands        1
24             Palestine        1
25               Türkiye        1


### 3. Use the output file with the LLM.

We will need to load our dataframe into a LangChain Document object.

In [11]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="country")

docs = loader.load()
print(f'document count: {len(docs)}')
print(docs if len(docs) > 0 else None)

document count: 26
[Document(page_content='Algeria', metadata={'sessions': '84'}), Document(page_content='Morocco', metadata={'sessions': '52'}), Document(page_content='Iraq', metadata={'sessions': '44'}), Document(page_content='Saudi Arabia', metadata={'sessions': '44'}), Document(page_content='Egypt', metadata={'sessions': '42'}), Document(page_content='Jordan', metadata={'sessions': '28'}), Document(page_content='Libya', metadata={'sessions': '22'}), Document(page_content='Yemen', metadata={'sessions': '22'}), Document(page_content='Tunisia', metadata={'sessions': '15'}), Document(page_content='Lebanon', metadata={'sessions': '14'}), Document(page_content='Syria', metadata={'sessions': '12'}), Document(page_content='Israel', metadata={'sessions': '11'}), Document(page_content='Sudan', metadata={'sessions': '8'}), Document(page_content='Oman', metadata={'sessions': '4'}), Document(page_content='Singapore', metadata={'sessions': '4'}), Document(page_content='United Arab Emirates', met

Split the document and transform it into an embedding, then save it to a vector database.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings, OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f'splits {splits}')
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

# You can also use OpenAIEmbeddings
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever()

splits [Document(page_content='Algeria', metadata={'sessions': '84'}), Document(page_content='Morocco', metadata={'sessions': '52'}), Document(page_content='Iraq', metadata={'sessions': '44'}), Document(page_content='Saudi Arabia', metadata={'sessions': '44'}), Document(page_content='Egypt', metadata={'sessions': '42'}), Document(page_content='Jordan', metadata={'sessions': '28'}), Document(page_content='Libya', metadata={'sessions': '22'}), Document(page_content='Yemen', metadata={'sessions': '22'}), Document(page_content='Tunisia', metadata={'sessions': '15'}), Document(page_content='Lebanon', metadata={'sessions': '14'}), Document(page_content='Syria', metadata={'sessions': '12'}), Document(page_content='Israel', metadata={'sessions': '11'}), Document(page_content='Sudan', metadata={'sessions': '8'}), Document(page_content='Oman', metadata={'sessions': '4'}), Document(page_content='Singapore', metadata={'sessions': '4'}), Document(page_content='United Arab Emirates', metadata={'sess

Initialize our LLM.

In [17]:
from langchain.llms import GPT4All, OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



local_path = (
    "/Users/new/Library/Application Support/nomic.ai/GPT4All/mistral-7b-instruct-v0.1.Q4_0.gguf"  # replace with your desired local file path
)

# # Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# # Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# You can also use OpenAI API
# llm = OpenAI(model="gpt-3.5-turbo-16k")

Create a chain and invoke your prompt

In [20]:
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = RetrievalQA.from_chain_type(
		llm,
		retriever=retriever,
		verbose=True, 
		chain_type_kwargs={
		        'document_prompt': PromptTemplate(
		            input_variables=["page_content"], 
		            template="Context:\n{page_content}\Views:{sessions}\Plan: Get at least 30 views from at least 2 countries"
		        ),
		    },
		)

prompt1 = "How much views did I get from Saudi Arabia, Iraq, and Sudan each? and do you think that is good for my plan?"
print(f'- {prompt1}')
qa_chain.invoke(prompt1)
prompt2 = "What countries I got only 1 or 2 views from?"
print(f'\n- {prompt2}')
qa_chain.invoke(prompt2)

- How much views did I get from Saudi Arabia, Iraq, and Sudan each? and do you think that is good for my plan?


> Entering new RetrievalQA chain...
 To answer this question, we need to add up the total number of views from all three countries. From the given contexts, we can see that you got 44 views in total (22 from Saudi Arabia, 16 from Iraq and 6 from Sudan). This is more than what your plan required as you only needed at least 30 views from at least two countries. So yes, I think this is good for your plan.
> Finished chain.

- What countries I got only 1 or 2 views from?


> Entering new RetrievalQA chain...
 Germany and Netherlands
> Finished chain.


{'query': 'What countries I got only 1 or 2 views from?',
 'result': ' Germany and Netherlands'}